### GPS Tracking demo for MS-Dessi

Let's collect gps sensor data in one data 
frame

In [45]:
import pandas as pd
from datetime import datetime

lat_file = pd.read_csv("data_latitude_016b.csv")
long_file = pd.read_csv("data_longitude_016b.csv")
df = pd.merge(lat_file, long_file, on="time")
df = df.rename(columns={"latitude_016b": "latitude", "longitude_016b": "longitude"})
df  = df.drop(['mean_x', 'mean_y'], axis=1)
df['date'] = pd.to_datetime(df['time'], format='mixed').dt.date
print(df.head())
print('\nNum non-None Longitude records: ' + str(len(df[~pd.isnull(df.longitude)])))
print('\nNum of unique dates: ' + str( len(df.date.unique())))
df_15sept = df[df.date == datetime.strptime("2023-09-15", '%Y-%m-%d').date()]
print('\nNum of records on the 15th of september: ' + str(len(df[df.date == datetime.strptime("2023-09-15", '%Y-%m-%d').date()])))

                               time   latitude  longitude        date
0  2023-09-18 16:28:09.420000+00:00  56.649734  16.462860  2023-09-18
1  2023-09-17 16:26:14.408000+00:00  56.649732  16.462823  2023-09-17
2  2023-09-16 16:25:10.163000+00:00  56.650230  16.463026  2023-09-16
3  2023-09-15 16:22:09.640000+00:00  56.650230  16.463026  2023-09-15
4  2023-09-15 16:17:09.888000+00:00  56.652808  16.443114  2023-09-15

Num non-None Longitude records: 85

Num of unique dates: 19

Num of records on the 15th of september: 38


In [ ]:
%Visualising all available points on the map 

In [3]:
%matplotlib inline

In [48]:
import folium   # (https://pypi.python.org/pypi/folium)
df = df_15sept
mymap = folium.Map( location=[ df.latitude.mean(), df.longitude.mean() ], zoom_start=14)
#folium.PolyLine(df[['Latitude','Longitude']].values, color="red", weight=2.5, opacity=1).add_to(mymap)
for coord in df[['latitude','longitude']].values:
    folium.CircleMarker(location=[coord[0],coord[1]], radius=1,color='red').add_to(mymap)
mymap   # shows map inline in Jupyter but takes up full width
#mymap.save('fol.html')  # saves to html file for display below

/Users/oxana/projects/ms-dessi/dessi/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [47]:
# calculate distances on surface of ellipsoid on the 15th of sept
from vincenty import vincenty

df['lastLat']=df['latitude'].shift(1)
df['lastLong']=df['longitude'].shift(1)
df['dist(meters)'] = df.apply(lambda x: vincenty((x['latitude'], x['longitude']), (x['lastLat'], x['lastLong'])), axis = 1) * 1000.

print('Total distance as summed between points in track:')
print('   ' + str(sum(df['dist(meters)'][1:])*0.001) + ' km')
# The df['dist'][1:] above is because the "shift" sets the first lastLon,lastLat as NaN.


Total distance as summed between points in track:
   58.592503999999984 km
